In [108]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')

import warnings
warnings.filterwarnings("ignore")

from myutil import *
from features import *

df = pd.read_csv('data/data.csv')

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

df['risk_score_t'] = df['risk_score_t'].replace(0.0, 0.01)
df['g_continuous'] = df['g_continuous'].replace(0.0, 1)
# df = df[(df['risk_score_t'] > 0.1) & (df['g_continuous'] > 0.1)]
# subset a sample of 5000 rows of df
# df = df.sample(n=1000, random_state=1)
# df.shape

all_features = df[get_all_features(df)]
feats = all_features.values
risk = df['risk_score_t'].values
gainF = df['g_continuous'].values
decision = df['propensity_score'].values
cost = np.ones(risk.shape)

alpha = 0.5
Q = 1e3

utility = risk * gainF * decision

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class RiskDataset(Dataset):
    def __init__(self, features, risks):
        self.features = torch.FloatTensor(features)
        self.risks = torch.FloatTensor(risks).reshape(-1, 1)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.risks[idx]
    
class RiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),  # Ensure the output is non-negative
            nn.Linear(64, 32),
            nn.ReLU(),  # Ensure the output is non-negative
            nn.Linear(32, 1),
            nn.Softplus()  # Ensure the output is non-negative
        )
    
    def forward(self, x):
        return self.model(x)

# Training function
def train_model(features, risks, epochs=10, batch_size=32):
    dataset = RiskDataset(features, risks)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = RiskPredictor(features.shape[1])
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(epochs):
        for batch_features, batch_risks in dataloader:
            optimizer.zero_grad()
            predictions = model(batch_features)
            loss = criterion(predictions, batch_risks)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model


model = train_model(feats, risk)

# predict the risk
model.eval()
predicted_risk = model(torch.FloatTensor(feats)).detach().numpy()

predicted_risk.mean()

Epoch [5/10], Loss: 5.2280
Epoch [10/10], Loss: 9.7929


5.384743

In [109]:
df['risk_score_t'].describe()

count    48784.000000
mean         4.393733
std          5.519548
min          0.010000
25%          1.443859
50%          2.887719
75%          5.350773
max        100.000000
Name: risk_score_t, dtype: float64

In [110]:
predicted_risk.std(), predicted_risk.min(), predicted_risk.max()

(4.900792, 0.0, 100.05635)

# Solve Optimization

In [111]:
def AlphaFairness(util, alpha):
    """
    Compute the alpha-fairness objective given a vector of utilities.
    util = array of utilities (e.g. r_i*g_i*d_i)
    alpha can be a positive float, 0, 1, or 'inf'.
    """
    if alpha == 1:
        # Proportional fairness
        # We assume util > 0 to avoid log(0)
        return np.sum(np.log(util + 1e-16))  # small offset to avoid log(0)
    elif alpha == 0:
        # Sum of utilities
        return np.sum(util)
    elif alpha == 'inf':
        # Max-min fairness
        return np.min(util)
    else:
        # General alpha > 0 (excluding alpha=1 handled above)
        return np.sum(util**(1 - alpha) / (1 - alpha))


def solve_optimization(gainF, risk, cost, alpha, Q):
    # Flatten input arrays
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)
    
    utils = cp.multiply(cp.multiply(gainF, risk), d)
    
    if alpha == 'inf':
        # Maximin formulation
        t = cp.Variable()  # auxiliary variable for minimum utility
        objective = cp.Maximize(t)
        constraints = [
            d >= 0.01,
            d <= 1,
            cost * d <= Q,
            utils >= t  # t is the minimum utility
        ]
    elif alpha == 1:
        # Nash welfare (alpha = 1)
        objective = cp.Maximize(cp.sum(cp.log(utils)))
        constraints = [
            d >= 0.01,
            d <= 1,
            cost * d <= Q
        ]
    elif alpha == 0:
        # Utilitarian (alpha = 0)
        objective = cp.Maximize(cp.sum(utils))
        constraints = [
            d >= 0.01,
            d <= 1,
            cost * d <= Q
        ]
    else:
        # General alpha-fairness
        objective = cp.Maximize(cp.sum(utils**(1-alpha))/(1-alpha) if alpha != 0 
                              else cp.sum(utils))
        constraints = [
            d >= 0.01,
            d <= 1,
            cost * d <= Q
        ]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False)
    
    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")
    
    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)
    
    return optimal_decision, optimal_value



def solve_alpha_fairness_closed_form(gainF, risk, cost, alpha, Q):
    """
    Solve the alpha-fairness problem in closed form or via direct logic:
      maximize   sum_i [ (r_i * g_i * d_i)^(1-alpha) / (1-alpha) ]    for alpha>0, alpha!=1
      subject to sum_i c_i * d_i <= Q,
                 0 <= d_i <= 1.
    Special cases:
      alpha = 0:    sum of r_i*g_i*d_i   (linear / "utilitarian")
      alpha = 1:    sum of log(r_i*g_i*d_i)  (proportional fairness)
      alpha = inf:  min_{i} r_i*g_i*d_i  (max-min fairness)

    Parameters
    ----------
    gainF : np.array
        Gain factors g_i (must be nonnegative).
    risk : np.array
        Risk factors r_i (must be positive).
    cost : np.array
        Cost factors c_i (must be nonnegative).
    alpha : float, 0, 1, or 'inf'
        Alpha parameter for alpha-fairness.
    Q : float
        Budget such that sum_i c_i * d_i <= Q.

    Returns
    -------
    d_opt : np.array
        The optimal decision vector in [0,1].
    objective_value : float
        The alpha-fairness objective achieved at d_opt.
    """

    gainF = gainF.flatten()
    risk = risk.flatten()
    cost = cost.flatten()
    n = len(risk)

    # Quick checks
    if np.any(cost < 0):
        raise ValueError("Cost must be nonnegative.")
    if np.any(risk < 0):
        raise ValueError("Risk must be nonnegative.")
    if np.any(gainF < 0):
        raise ValueError("Gain factors must be nonnegative.")

    # Handle corner cases: alpha=0, alpha=1, alpha='inf'
    # --------------------------------------------------
    if alpha == 0:
        # We want to maximize sum_i r_i*g_i*d_i subject to sum_i c_i*d_i <= Q and 0<=d_i<=1.
        # This is a "linear knapsack" with partial usage allowed (since d_i up to 1).
        # Sort by highest ratio (r_i*g_i)/c_i and fill until budget used or d_i=1.
        
        ratio = np.zeros(n)
        for i in range(n):
            if cost[i] > 0:
                ratio[i] = (risk[i] * gainF[i]) / cost[i]
            else:
                # If cost[i] == 0 but gain is positive, we set d_i=1 for free
                ratio[i] = 1e15 if (risk[i]*gainF[i])>0 else 0.0

        sorted_idx = np.argsort(-ratio)  # descending order
        d_opt = np.zeros(n)
        budget_left = Q

        for i in sorted_idx:
            if budget_left <= 1e-15:
                break
            # Max we could allocate is d_i=1 => cost[i]*1 = cost[i]
            if cost[i] <= budget_left:  
                # We can afford to set d_i=1
                d_opt[i] = 1.0
                budget_left -= cost[i]
            else:
                # We only partially fill up to use the remaining budget
                if cost[i] > 1e-15:
                    frac = budget_left / cost[i]
                    if frac > 1.0:
                        frac = 1.0
                    d_opt[i] = frac
                    budget_left -= cost[i] * frac

        objective_value = AlphaFairness(risk * gainF * d_opt, alpha=0)
        return d_opt, objective_value

    elif alpha == 1:
        # Proportional fairness: sum_i log(r_i*g_i*d_i).
        # There's no direct simple closed-form for box constraints, 
        # but the classical unconstrained solution (only budget constraint) 
        # leads to d_i ∝ 1 / (lambda*c_i). Then we must "clip" or "waterfill" to handle d_i <= 1.
        #
        # We do an iterative approach: if some d_i > 1, fix it to 1, remove from the "free" set, re-solve for others.
        
        # Initialize set of free indices
        free_idx = list(range(n))
        d_opt = np.zeros(n)
        budget_left = Q

        while True:
            # For free indices, cost sum must be budget_left
            # if we treat partial solution: d_i = x / (c_i) => sum_i c_i*d_i = x * sum_i 1 => that's not quite it
            #
            # Actually for alpha=1 (proportional fairness), ignoring the d<=1 constraint,
            # the known result is d_i = k / c_i, for some k>0, s.t. sum_i c_i * (k/c_i) = k * sum_i (c_i/c_i) = k*n = Q => k = Q / n.
            # But that doesn't incorporate r_i*g_i. 
            #
            # The more standard method (with an extra weighting w_i=r_i*g_i) would come from setting
            # ∂/∂d_i [ sum_j log(w_j*d_j) ]=0 => ∑ log(w_j*d_j) => for i, 1/(w_i*d_i)* w_i = λ c_i => 1/d_i = λ c_i => d_i = 1/(λ c_i).
            # Then sum_i c_i * d_i = sum_i c_i * [1/(λ c_i)] = sum_i [1/λ] = n/λ => λ = n / Q. 
            # This is if r_i*g_i = constant across i. If r_i*g_i differ, we get 1/(d_i) = λ c_i / (r_i*g_i). 
            # => d_i = (r_i*g_i)/(λ c_i).
            #
            # So let's incorporate r_i*g_i: from stationarity,
            #    ∂/∂d_i [ sum_j log(r_j*g_j*d_j) ] = (1 / (r_i*g_i*d_i)) * (r_i*g_i) = 1/d_i => set = λ c_i => d_i = 1/(λ c_i)
            # but we missed the factor (r_i*g_i). Actually if we do it carefully:
            #    partial wrt d_i => (r_i*g_i)/(r_i*g_i*d_i) = 1/d_i => yes, it doesn't incorporate r_i*g_i in the final ratio. 
            # That suggests the unconstrained solution for alpha=1 doesn't depend on r_i*g_i at all for the budget constraint. 
            # It's the classical result that leads to "equal share of budget" if cost_i are identical, ignoring r_i*g_i. 
            #
            # Then sum_i c_i * d_i = Q => sum_i c_i*(1/(λ c_i)) = sum_i(1/λ)=n/λ => λ = n/Q. 
            # => d_i = Q/(n*c_i). 
            #
            # Next step: if d_i>1 for some i, fix d_i=1, remove it from free set, reduce budget, recalc for others. 
            #
            # Let's do that loop:

            if len(free_idx) == 0:
                break

            n_free = len(free_idx)
            # "lambda" = n_free / budget_left
            # => d_i = budget_left / (n_free * c_i)
            d_proposed = np.zeros(n_free)
            for k, i in enumerate(free_idx):
                d_proposed[k] = budget_left / (n_free * cost[i]) if cost[i] > 1e-15 else 1.0e15

            # Check which exceed 1
            exceed_mask = (d_proposed > 1.0)
            if not np.any(exceed_mask):
                # None exceed 1 => accept them
                for k, i in enumerate(free_idx):
                    d_opt[i] = d_proposed[k]
                break
            else:
                # Some indices want to be > 1 => fix them to 1, remove from free set
                # We'll fix all that exceed 1 simultaneously
                still_free_idx = []
                for k, i in enumerate(free_idx):
                    if d_proposed[k] > 1:
                        d_opt[i] = 1.0
                        # reduce budget
                        budget_left -= cost[i] * 1.0
                    else:
                        still_free_idx.append(i)
                free_idx = still_free_idx

        # Evaluate objective
        util = risk * gainF * d_opt
        objective_value = AlphaFairness(util, alpha=1)
        return d_opt, objective_value

    elif alpha == 'inf':
        # Max-min fairness: maximize min_i {r_i*g_i*d_i}
        # subject to sum_i c_i*d_i <= Q, 0 <= d_i <= 1
        #
        # There's a known approach: let d_i = t / (r_i*g_i), sum_i c_i * (t / (r_i*g_i)) <= Q => t <= Q / sum_i (c_i / (r_i*g_i)).
        # Then if that yields d_i > 1 for some i, fix d_i=1, remove i from the sum, re-solve for the rest. Water-filling approach.

        # Start with all free
        free_idx = list(range(n))
        d_opt = np.zeros(n)
        budget_left = Q

        while True:
            if len(free_idx) == 0:
                break

            # Solve for a common t => d_i = t/(r_i*g_i)
            denom = 0.0
            for i in free_idx:
                # sum_i c_i*(t/(r_i*g_i)) = t * sum_i [ c_i/(r_i*g_i) ]
                denom += cost[i] / (risk[i]*gainF[i]) if (risk[i]*gainF[i] > 1e-15) else 1e15
            if denom < 1e-15:
                # Degenerate case (maybe cost=0 or r*g=0 for all in free set)
                # Then we just fill them with d=1 if that doesn't break budget or 0 otherwise
                for i in free_idx:
                    if cost[i] <= budget_left:
                        d_opt[i] = 1.0
                        budget_left -= cost[i]
                free_idx = []
                break

            t = budget_left / denom

            # Check which d_i = t/(r_i*g_i) would exceed 1
            new_fixed = []
            for i in free_idx:
                if risk[i]*gainF[i] < 1e-15:
                    # If r_i*g_i=0, then max-min approach is 0 anyway => might as well skip
                    d_candidate = 0.0
                else:
                    d_candidate = t / (risk[i]*gainF[i])
                if d_candidate > 1.0:
                    new_fixed.append(i)

            if len(new_fixed) == 0:
                # All d_i <= 1, accept them
                for i in free_idx:
                    d_opt[i] = t / (risk[i]*gainF[i]) if (risk[i]*gainF[i]>1e-15) else 0.0
                break
            else:
                # Fix those above 1 to d=1
                for i in new_fixed:
                    d_opt[i] = 1.0
                    # reduce budget
                    budget_left -= cost[i]
                # remove them from free set
                free_idx = [i for i in free_idx if i not in new_fixed]

        util = risk * gainF * d_opt
        objective_value = AlphaFairness(util, alpha='inf')
        return d_opt, objective_value

    # General case: alpha>0, alpha!=1
    # --------------------------------
    # Closed form ignoring d_i <=1: 
    #   d_i^* = 
    #     \left( (r_i*g_i)^{1-alpha} / [\lambda c_i] \right)^{1/alpha}
    #   Then fix lambda by sum_i c_i d_i^* = Q
    #
    #   Let X_i = c_i^{-1/alpha} * (r_i*g_i)^{(1-alpha)/alpha} = c_i^{-1/alpha} * (r_i*g_i)^{1/alpha -1}
    #   Then sum_i c_i[K * X_i] = K * sum_i c_i X_i = Q => K= Q / sum_i c_i X_i
    #   => d_i^unconstr = K * X_i
    #
    # If d_i^unconstr > 1, we saturate d_i=1, remove from free set, reduce budget, re-solve for others, repeat.

    if alpha <= 0:
        raise ValueError("alpha must be positive (except for alpha=0 handled earlier).")

    # Initialize free set (indices not saturated to 1)
    free_idx = list(range(n))
    d_opt = np.zeros(n)
    budget_left = Q

    while True:
        if len(free_idx) == 0:
            break

        # Compute X_i = c_i^(-1/alpha) * (r_i*g_i)^{1/alpha - 1}
        X = np.zeros(len(free_idx))
        for k, i in enumerate(free_idx):
            if cost[i] < 1e-15:
                # cost=0 => we can set d_i=1 for free if (r_i*g_i)>0
                # treat c_i^-1/alpha as large => let's just handle it as a special case
                X[k] = 1e20 if (risk[i]*gainF[i] > 1e-15) else 0.0
            else:
                val = (risk[i]*gainF[i])**(1.0/alpha - 1.0)
                X[k] = (cost[i]**(-1.0/alpha)) * val
        
        denom = 0.0
        for k, i in enumerate(free_idx):
            denom += cost[i]*X[k]

        if denom < 1e-15:
            # Something degenerate => push everything to d=1 if possible
            for i in free_idx:
                if cost[i] <= budget_left:
                    d_opt[i] = 1.0
                    budget_left -= cost[i]
            free_idx = []
            break

        K = budget_left / denom

        # Unconstrained solution
        d_candidate = np.zeros(len(free_idx))
        for k, i in enumerate(free_idx):
            d_candidate[k] = K * X[k]

        # Check which exceed 1
        exceed_mask = (d_candidate > 1.0)
        if not np.any(exceed_mask):
            # All good => accept them
            for k, i in enumerate(free_idx):
                d_opt[i] = d_candidate[k]
            break
        else:
            # Some saturate to 1
            new_fixed = [ free_idx[j] for j in range(len(free_idx)) if exceed_mask[j] ]
            for i in new_fixed:
                d_opt[i] = 1.0
                budget_left -= cost[i]
            # Keep those not saturating
            free_idx = [ free_idx[j] for j in range(len(free_idx)) if not exceed_mask[j] ]

    util = risk * gainF * d_opt
    objective_value = AlphaFairness(util, alpha)
    return d_opt, objective_value


In [112]:
data_sample = df.sample(n=10000, random_state=1)
feats_sample = data_sample[get_all_features(data_sample)].values
risk_sample = data_sample['risk_score_t'].values
gainF_sample = data_sample['g_continuous'].values
decision_sample = data_sample['propensity_score'].values
cost_sample = np.ones(risk_sample.shape)

alpha = 0.5
Q = 500

def printStats(data):
    print(f"Mean: {np.mean(data):.2f}")
    print(f"Median: {np.median(data):.2f}")
    print(f"Std. Dev.: {np.std(data):.2f}")
    print(f"Min: {np.min(data):.2f}")
    print(f"Max: {np.max(data):.2f}")


In [113]:
predicted_risk_sample = model(torch.FloatTensor(feats_sample)).detach().numpy()
printStats(predicted_risk_sample)
# pred_sol, _ = solve_optimization(gainF_sample, predicted_risk_sample, cost_sample, alpha, Q)
# true_sol, true_obj = solve_optimization(gainF_sample, risk_sample, cost_sample, alpha, Q)

# pred_sol

Mean: 5.37
Median: 4.06
Std. Dev.: 4.84
Min: 0.01
Max: 87.65


In [ ]:
# Solve the alpha-fairness problem with true risk
alpha = 0.5
true_sol, _ = solve_optimization(gainF_sample, risk_sample, cost_sample, alpha, Q)
true_obj = AlphaFairness(true_sol * gainF_sample * risk_sample, alpha)

print(f"True solution: {true_sol}", f"Objective: {true_obj:.2f}", sep='\n')


# Solve the alpha-fairness problem with predicted risk
pred_sol, _ = solve_optimization(gainF_sample, predicted_risk_sample, cost_sample, alpha, Q)
pred_obj = AlphaFairness(pred_sol * gainF_sample * risk_sample, alpha)

print(f"Predicted solution: {pred_sol}", f"Objective: {pred_obj:.2f}", sep='\n')

print(f"regret : {true_obj - pred_obj}")

# Solve the alpha-fairness problem using closed-form with true risk and predicted risk
true_sol_closed, _ = solve_alpha_fairness_closed_form(gainF_sample, risk_sample, cost_sample, alpha, Q)
true_obj_closed = AlphaFairness(true_sol_closed * gainF_sample * risk_sample, alpha)

pred_sol_closed, _ = solve_alpha_fairness_closed_form(gainF_sample, predicted_risk_sample, cost_sample, alpha, Q)
pred_obj_closed = AlphaFairness(pred_sol_closed * gainF_sample * risk_sample, alpha)

print(f"True solution (closed-form): {true_sol_closed}", f"Objective: {true_obj_closed:.2f}", sep='\n')
print(f"Predicted solution (closed-form): {pred_sol_closed}", f"Objective: {pred_obj_closed:.2f}", sep='\n')
      
print(f"regret : {true_obj_closed - pred_obj_closed}")

True solution: [0.01 0.01 0.01 ... 0.01 1.   0.01]
Objective: 80981.38
Predicted solution: [0.01 0.01 1.   ... 0.01 0.01 0.01]
Objective: 60577.09
regret : 20404.29152476885
True solution (closed-form): [0. 0. 1. ... 0. 1. 0.]
Objective: 93230.15
Predicted solution (closed-form): [0. 0. 1. ... 0. 0. 0.]
Objective: 70211.24
regret : 23018.90746794225


In [115]:
# Is the closed form working?
# Check if the closed-form solution is correct
np.allclose(true_sol, true_sol_closed, atol=1e-3), np.allclose(pred_sol, pred_sol_closed, atol=1e-3)

(False, False)